## Module 4: Python


# Developing Efficiency
## DATABASES
<br>

Asel Kushkeyeva<br>
Data Science Institute, University of Toronto<br>
2022

### Jupyter Notebook as a Slideshow

To see this notebook as a live slideshow, we need to install RISE (Reveal.js - Jupyter/IPython Slideshow Extension):

1. Insert a cell and execute the following code: `conda install -c conda-forge rise`
2. Restart the Jupyter Notebook.
3. On the top of your notebook you have a new icon that looks like a bar chart; hover over the icon to see 'Enter/Exit RISE Slideshow'.
4. Click on the RISE icon and enjoy the slideshow.
5. You can edit the notebook in a slideshow mode by double clicking the line.
*This is done only once. Now all your notebooks will have the RISE extension (unless you re-install the Jupyter Notebook).*

# Agenda

1. Creating Databases
2. Updating and Deleting Databases
3. Joins to Combine Tables

# Databases

As we have learned in the Data Storage and Access module, SQL is the language to manipulate relational databases.

Today we will learn how Python works with SQL by closely following __Chapter 17. Databases__ of the key text *Practical Programming: An Introduction to Computer Science Using Python 3.6* 

In [2]:
import sqlite3 # an SQL package

In [38]:
# establish a conncetion with a database and get a cursor:
con = sqlite3.connect('population_1.db')
cur = con.cursor()

In [39]:
# SQL statement is to be put in a string
cur.execute('CREATE TABLE PopByRegion_1(Region TEXT, Population INTEGER)')

In [41]:
# Insert data in the PopByRegion table
cur.execute('INSERT INTO PopByRegion_1 VALUES("Central Africa", 330993)')
cur.execute('INSERT INTO PopByRegion_1 VALUES("Southeastern Africa", 743112)')

In [43]:
# insert the rest of the data
cur.execute('''INSERT INTO PopByRegion_1 VALUES 
('Southern Asia', 2051941), 
('Northern Africa', 1037463),
('Asia Pacific', 785468),
('Middle East', 687630),
('Eastern Asia', 1362955), 
('South America', 593121), 
('Eastern Europe', 223427), 
('North America', 661157), 
('Western Europe', 387933)
''')

# another way of inserting data:
cur.execute('INSERT INTO PopByRegion_1 VALUES (?, ?)', ("Japan", 100562))

In [44]:
# save the changes
con.commit()

In [45]:
# close connection
con.close()

After closing the connection to work in the database again, we need to establish the connection and get the cursor. Similarly, we need to run the `connect()` and `cursor()` after restarting the notebook.

In [3]:
# establish a conncetion with a database and get a cursor:
con = sqlite3.connect('population_1.db')
cur = con.cursor()

In [51]:
# run a query
cur.execute('SELECT Region, Population FROM PopByRegion_1')

In [52]:
# after running a query, return one record
cur.fetchone()

('Central Africa', 330993)

In [54]:
# return all leftover records
cur.fetchall()

[]

Once all the records are retrieved the following commands do not produce any output.

In [55]:
cur.fetchone()

In [56]:
cur.fetchall()

[]

Ordering the records:

In [58]:
cur.execute('SELECT Region, Population FROM PopByRegion_1 ORDER BY Region')

In [59]:
cur.fetchall()

[('Asia Pacific', 785468),
 ('Central Africa', 330993),
 ('Eastern Asia', 1362955),
 ('Eastern Europe', 223427),
 ('Japan', 100562),
 ('Middle East', 687630),
 ('North America', 661157),
 ('Northern Africa', 1037463),
 ('South America', 593121),
 ('Southeastern Africa', 743112),
 ('Southern Asia', 2051941),
 ('Western Europe', 387933)]

Query conditions:

In [61]:
cur.execute('SELECT Region FROM PopByRegion_1 WHERE Population > 1000000')

In [62]:
cur.fetchall()

[('Southern Asia',), ('Northern Africa',), ('Eastern Asia',)]

In [64]:
cur.execute('''SELECT Region FROM PopByRegion_1
WHERE Population > 1000000 AND Region < "L"''')

In [65]:
cur.fetchall()

[('Eastern Asia',)]

## Updating and Deleting

In [21]:
cur.execute('SELECT * FROM PopByRegion_1 WHERE Region = "Japan"')

In [22]:
cur.fetchall()

[('Japan', 100562)]

In [24]:
# change Japan's population value:
cur.execute('''
UPDATE PopByRegion_1 
SET Population = 100600
WHERE Region = "Japan"''')

In [25]:
# check if the update took place
cur.execute('SELECT * FROM PopByRegion_1 WHERE Region = "Japan"')
cur.fetchall()

[('Japan', 100600)]

A command to delete a table is as follows:

DROP TABLE table_name

In [26]:
# to delete individual records:
cur.execute('DELETE FROM PopByRegion_1 WHERE Region < "L"')

In [28]:
# see the changes
cur.execute('SELECT * FROM PopByRegion_1')
cur.fetchall()

[('Southeastern Africa', 743112),
 ('Southern Asia', 2051941),
 ('Northern Africa', 1037463),
 ('Middle East', 687630),
 ('South America', 593121),
 ('North America', 661157),
 ('Western Europe', 387933)]

In [30]:
# let us put the deleted record back in the table
cur.execute('INSERT INTO PopByRegion_1 VALUES ("Japan", 100562)')

## Joins to Combine Tables

In [4]:
# create another table:
cur.execute('''CREATE TABLE PopByCountry(Region TEXT, Country TEXT, Population INTEGER)''')

In [5]:
# insert data:
cur.execute('''INSERT INTO PopByCountry VALUES("Eastern Asia", "China",
1285238)''')

In [6]:
# more efficient insertion:
countries = [("Eastern Asia", "DPR Korea", 24056), 
             ("Eastern Asia", "Hong Kong (China)", 8764), 
             ("Eastern Asia", "Mongolia", 3407), 
             ("Eastern Asia", "Republic of Korea", 41491), 
             ("Eastern Asia", "Taiwan", 1433),
             ("North America", "Bahamas", 368), 
             ("North America", "Canada", 40876),
             ("North America", "Greenland", 43), 
             ("North America", "Mexico", 126875),
             ("North America", "United States", 493038)]

In [7]:
for c in countries:
    cur.execute('INSERT INTO PopByCountry VALUES (?, ?, ?)', (c[0], c[1], c[2]))

In [8]:
con.commit()

In [9]:
# join the tables:
cur.execute(''' 
SELECT PopByRegion_1.Region, PopByCountry.Country
FROM PopByRegion_1
INNER JOIN PopByCountry
WHERE (PopByRegion_1.Region = PopByCountry.Region)
AND (PopByRegion_1.Population > 1000000)
''')

In [10]:
cur.fetchall()

[('Eastern Asia', 'China'),
 ('Eastern Asia', 'DPR Korea'),
 ('Eastern Asia', 'Hong Kong (China)'),
 ('Eastern Asia', 'Mongolia'),
 ('Eastern Asia', 'Republic of Korea'),
 ('Eastern Asia', 'Taiwan')]

In [11]:
# to find the regions where one country accounts for more than 10 percent of the region’s overall population
cur.execute('''
SELECT PopByCountry.Region
FROM PopByCountry
INNER JOIN PopByRegion_1
WHERE (PopByCountry.Region = PopByRegion_1.Region)
AND (PopByCountry.Population > PopByRegion_1.Population * 0.10)
''')

In [12]:
cur.fetchall()

[('Eastern Asia',), ('North America',), ('North America',)]

In [13]:
# to remove duplicates:
cur.execute('''
SELECT DISTINCT PopByCountry.Region
FROM PopByCountry
INNER JOIN PopByRegion_1
WHERE (PopByCountry.Region = PopByRegion_1.Region)
AND (PopByCountry.Population > PopByRegion_1.Population * 0.10)
''')

In [14]:
cur.fetchall()

[('Eastern Asia',), ('North America',)]

### Self Join

To find pairs of countries in a table whose populations are close to each other — say, within 1,000 of each other.

In [31]:
cur.execute('''
SELECT A.Country, B.Country
FROM PopByCountry A
INNER JOIN PopByCountry B
WHERE (ABS(A.Population - B.Population) <= 1000)
AND A.Country != B.Country''')

In [32]:
cur.fetchall()

[('Republic of Korea', 'Canada'),
 ('Bahamas', 'Greenland'),
 ('Canada', 'Republic of Korea'),
 ('Greenland', 'Bahamas')]

Keys and other constraints should be added at the time of table creation.

__Agregate Functions:__

SUM, AVG, MIN, MAX, COUNT

In [33]:
# grouping
cur.execute('''SELECT Region, SUM (Population) FROM PopByCountry
GROUP BY Region''')

In [34]:
cur.fetchall()

[('Eastern Asia', 1364389), ('North America', 661200)]

### Nested Queries

To write a query on the PopByCountry table to get the regions that do not have a country with a population of 8,764,000.

In [35]:
cur.execute('''
SELECT DISTINCT Region
FROM PopByCountry
WHERE Region NOT IN
(SELECT DISTINCT Region
FROM PopByCountry
WHERE (PopByCountry.Population = 8764))
''')

In [36]:
cur.fetchall()

[('North America',)]

## PRACTICE IN YOUR NOTEBOOK

__Exercise 1 on page 365 *Practical Programming: an Introduction to Computer Science using Python 3.6*__

In this exercise, you will create a table to store the population and land area of the Canadian provinces and territories according to the 2001 census. Our data is taken from http://www12.statcan.ca/english/census01/home/index.cfm.

Province/Territory | Population | Land Area
-------------------|------------|--------
Newfoundland and Labrador | 512930 | 370501.69
Prince Edward Island | 135294 | 5684.39
Nova Scotia | 908007 | 52917.43
New Brunswick | 729498 | 71355.67
Quebec | 7237479 | 1357743.08
Ontario | 11410046 | 907655.59
Manitoba | 1119583 | 551937.87
Saskatchewan | 978933 | 586561.35
Alberta | 2974807 | 639987.12
British Columbia | 3907738 | 926492.48
Yukon Territory | 28674 | 474706.97
Northwest Territories | 37360 | 1141108.37
Nunavut | 26745 | 1925460.18


Write Python code that does the following:

a. Creates a new database called census.db

b. Makes a database table called Density that will hold the name of the province or territory (TEXT), the population (INTEGER), and the land area (REAL)

c. Inserts the data from the table above

e. Retrieves the populations

f. Retrieves the provinces that have populations of less than one million

g. Retrieves the provinces that have populations of less than one million or greater than five million

h. Retrieves the provinces that do not have populations of less than one million or greater than five million

i. Retrieves the populations of provinces that have a land area greater than 200,000 square kilometers

j. Retrieves the provinces along with their population densities (population divided by land area)

# References

- Chapter 17, Gries, Campbell, and Montojo, 2017, *Practical Programming: An Introduction to Computer Science Using Python 3.6*